In [90]:
import pygame
pygame.init()

(5, 0)

In [91]:
screen=pygame.display.set_mode((800,600))
pygame.display.set_caption("TicTacToe")
ButtonImg=pygame.image.load('square.png')

In [92]:
class Button():
	def __init__(self, x, y, image, scale):
		width = image.get_width()
		height = image.get_height()
		self.image = pygame.transform.scale(image, (int(width * scale), int(height * scale)))
		self.rect = self.image.get_rect()
		self.rect.topleft = (x, y)
		self.clicked = False

	def draw(self, surface):
		action = False
		#get mouse position
		pos = pygame.mouse.get_pos()

		#check mouseover and clicked conditions
		if self.rect.collidepoint(pos):
			if pygame.mouse.get_pressed()[0] == 1 and self.clicked == False:
				self.clicked = True
				action = True

		if pygame.mouse.get_pressed()[0] == 0:
			self.clicked = False

		#draw button on screen
		surface.blit(self.image, (self.rect.x, self.rect.y))
		return action

In [93]:
class Node:
    def __init__(self,x,y):
        self.x=x
        self.y=y
        self.button=Button(x,y,ButtonImg,1)
        self.lett=''
    def set_lett(self,val):
        pressed=self.button.draw(screen)
        if pressed and self.lett=='':
            self.lett=val
            return True
        return False
    def display_char(self,screen):
        pressed=self.button.draw(screen)
        if self.lett!='':
            font = pygame.font.SysFont('Verdana', 40, bold=True)
            lett_dis = font.render(self.lett, True, (0, 0, 0))
            lett_dis_rect = lett_dis.get_rect(center=(self.x+32, self.y+32))
            screen.blit(lett_dis, lett_dis_rect)


In [94]:
class Board:
    def __init__(self):
        self.nodes=[[Node(288,188),Node(368,188),Node(448,188)],[Node(288,268),Node(368,268),Node(448,268)],[Node(288,348),Node(368,348),Node(448,348)]]
        self.play=0
        self.turn=0
    def checkwin(self):
        for row in self.nodes:
            if row[0].lett == row[1].lett == row[2].lett != '':
                return row[0].lett
        for col in range(3):
            if self.nodes[0][col].lett == self.nodes[1][col].lett == self.nodes[2][col].lett != '':
                return self.nodes[0][col].lett
        if self.nodes[0][0].lett == self.nodes[1][1].lett == self.nodes[2][2].lett != '':
            return self.nodes[0][0].lett
        if self.nodes[0][2].lett == self.nodes[1][1].lett == self.nodes[2][0].lett != '':
            return self.nodes[0][2].lett
        count=0
        for i in range(3):
            for j in range(3):
                if self.nodes[i][j].lett!='':
                    count+=1
        if count==9:
            return 'Draw'
        return None
    
    def update_board(self,val):
        ans=False
        for i in range(3):
            for j in range(3):
                ans=ans or self.nodes[i][j].set_lett(val)
        return ans
    def reset(self):
        self.play = 1
        self.turn=0
        for i in range(3):
            for j in range(3):
                self.nodes[i][j].lett = ''
    def winner(self):
        k=self.checkwin()
        if k!=None:
            if k=='X':
                self.play=2
            if k=='O':
                self.play=3
            if k=='Draw':
                self.play=4  

    def display(self,screen):
        for i in range(3):
            for j in range(3):
                self.nodes[i][j].display_char(screen)


In [95]:
def display_menu(Board):
    font = pygame.font.SysFont('Verdana', 40, bold=True)
    if Board.play==0:
        text_surface = font.render("TicTacToe!", True, (0, 0, 0)) 
        text_rect = text_surface.get_rect(center=(400, 300))
        screen.blit(text_surface, text_rect) # yellow
    elif Board.play==2 or Board.play==3 or Board.play==4:
        if Board.play==4:
            text_surface = font.render("DRAW!", True, (0, 0, 0))
            text_rect = text_surface.get_rect(center=(400, 300))
            screen.blit(text_surface, text_rect)
        elif Board.play==2:
            text_win = font.render("X WON!", True, (0, 0, 0))
            text_win_rect = text_win.get_rect(center=(400, 350))
            screen.blit(text_win, text_win_rect)
        else:
            text_lose = font.render("O WON!", True, (0, 0, 0))
            text_lose_rect = text_lose.get_rect(center=(400, 350))
            screen.blit(text_lose, text_lose_rect)
    buttonImg=pygame.image.load('start-button.png')
    button=Button(350,450,buttonImg,0.8)
    k=button.draw(screen)
    if k:
        Board.reset()
        Board.play=1;

In [96]:
def AIPlayermove(board):
    maxi=-float('inf')
    bestmove=(-1,-1)
    for i in range(3):
        for j in range(3):
            if board.nodes[i][j].lett=='':
                board.nodes[i][j].lett='O';
                score=minimax(board,1)
                board.nodes[i][j].lett='';
                if score>maxi:
                    maxi=score
                    bestmove=(i,j)
    if bestmove != (-1, -1):
        i, j = bestmove
        board.nodes[i][j].lett = 'O'
        return True
    return False

def minimax(board,depth):
    result=board.checkwin()
    if result is not None:
        if result=='X':
            return -1
        if result=='O':
            return 1
        if result=='Draw':
            return 0
    if depth%2==0:
        maxi=-float('inf')
        for i in range(3):
            for j in range(3):
                if board.nodes[i][j].lett=='':
                    board.nodes[i][j].lett='O';
                    score=minimax(board,depth+1)
                    board.nodes[i][j].lett='';
                    if score>maxi:
                        maxi=score
        return maxi
    if depth%2==1:
        mini=float('inf')
        for i in range(3):
            for j in range(3):
                if board.nodes[i][j].lett=='':
                    board.nodes[i][j].lett='X';
                    score=minimax(board,depth+1)
                    board.nodes[i][j].lett='';
                    if score<mini:
                        mini=score
        return mini
    return None

In [97]:
board=Board()
running=True
while running:
    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            running=False
    screen.fill((255,255,255))
    if board.play==1:
        if board.turn%2==0:
            k=board.update_board('X')
            if k:
                board.turn+=1
        else:
            k=AIPlayermove(board)
            if k:
                board.turn+=1
        board.display(screen)
        board.winner()
    else:
        display_menu(board)
    pygame.display.update()
pygame.quit()
